## Moderation Analysis

In [1]:
suppressWarnings(suppressMessages(library(tidyverse)))
suppressWarnings(suppressMessages(library(gridExtra)))
suppressWarnings(suppressMessages(library(memisc)))
library(emmeans)

# Load the data
load(file = "~/Documents/vcf/data/moderation.RData")
paste0("Initial sample: ", nrow(vcf_data))

[1] "Initial sample: 371"

### PHI Trust Interactions

In [2]:
test_interaction_phi_trust <- function(data){
    
    data$group <- factor(data$group, levels = c("Control", "Treatment"))    
    
    # fit base model    
    base_model <- aov(post ~ group*phi_trust_cat + wave + 
                      republican + conservative + 
                      employment2 + education2 + marital_status + gender + age + income2 +                     
                      pre, data = data)
     
    # calculate Box-Cox lambda
    bc <- trafo::boxcox(base_model, method = "ml", lambdarange = c(-10, 3), plotit = FALSE)
    lambda <- bc$lambdahat
    bctrans <- make.tran("boxcox", lambda)
  
    # fit model to transformed data    
    model <- with(bctrans,
              aov(linkfun(post) ~ group*phi_trust_cat + wave + 
                                  republican + conservative + 
                                  employment2 + education2 + marital_status + gender + age + income2 +                     
                                  linkfun(pre), data = data))       
    
     lm_model <- with(bctrans,
              lm(linkfun(post) ~ group*phi_trust_cat + wave + 
                                  republican + conservative + 
                                  employment2 + education2 + marital_status + gender + age + income2 +                     
                                  linkfun(pre), data = data))    
    
    mean <- model$coefficients[names(model$coefficients) %>% str_detect(":")]
    ci_tbl <- model %>% confint() %>% as_tibble()
    row_names <- model %>% confint() %>% rownames()
    idx <- which(row_names %>% str_detect(":"))
    low <- ci_tbl$`2.5 %`[idx]
    high <- ci_tbl$`97.5 %`[idx]
    idx <- which(rownames(car::Anova(model, type = "III")) %>% str_detect(":"))
    p <- car::Anova(model, type = "III")[[4]][idx]
    sig = ""
    if (p < 0.05){sig = "*"}
    if (p < 0.01){sig = "**"}
    if (p < 0.001){sig = "***"}
    ci_summary <- paste0("mean = ", round(mean, 3), 
                         "; 95% CI = ", round(low, 3), ", ", round(high, 3), 
                         "; P = ", round(p, 4))
        
    return(list(
        "base_model" = base_model,        
        "model" = model,
        "lm_model" = lm_model,
        "ci_summary" = ci_summary))}

In [3]:
intention <- test_interaction_phi_trust(data = vcf_data %>% dplyr::mutate(post = intention_6_n, pre = base_intention_n))
encourage <- test_interaction_phi_trust(data = vcf_data %>% dplyr::mutate(post = encourage_6_n, pre = base_encourage_n))
sp_confidence <- test_interaction_phi_trust(data = vcf_data %>% dplyr::mutate(post = sp_confidence_6_n, pre = base_sp_confidence_n))
sp_complacency <- test_interaction_phi_trust(data = vcf_data %>% dplyr::mutate(post = sp_complacency_6_n, pre = base_sp_complacency_n))
#car::Anova(intention$model, type = "III") %>% round(3)

paste0("Intentions: ", intention$ci_summary)
paste0("Intentions to encourage: ", encourage$ci_summary)
paste0("Vaccine confidence beliefs: ", sp_confidence$ci_summary)
paste0("Vaccine complacency beliefs: ", sp_complacency$ci_summary)

[1] "Intentions: mean = 0.103; 95% CI = -0.003, 0.21; P = 0.0578"

[1] "Intentions to encourage: mean = 0.127; 95% CI = -0.019, 0.272; P = 0.0878"

[1] "Vaccine confidence beliefs: mean = 0.214; 95% CI = 0.044, 0.384; P = 0.0136"

[1] "Vaccine complacency beliefs: mean = 0.029; 95% CI = -0.163, 0.222; P = 0.7631"

### Gov Trust Interactions

In [4]:
test_interaction_gov_trust <- function(data){
    
    data$group <- factor(data$group, levels = c("Control", "Treatment"))    
    
    # fit base model    
    base_model <- aov(post ~ group*gov_trust_cat + wave + 
                      republican + conservative + 
                      employment2 + education2 + marital_status + gender + age + income2 +                     
                      pre, data = data)
     
    # calculate Box-Cox lambda
    bc <- trafo::boxcox(base_model, method = "ml", lambdarange = c(-10, 3), plotit = FALSE)
    lambda <- bc$lambdahat
    bctrans <- make.tran("boxcox", lambda)
  
    # fit model to transformed data    
    model <- with(bctrans,
              aov(linkfun(post) ~ group*gov_trust_cat + wave + 
                                  republican + conservative + 
                                  employment2 + education2 + marital_status + gender + age + income2 +                     
                                  linkfun(pre), data = data)) 
    
    lm_model <- with(bctrans,
              lm(linkfun(post) ~ group*gov_trust_cat + wave + 
                                  republican + conservative + 
                                  employment2 + education2 + marital_status + gender + age + income2 +                     
                                  linkfun(pre), data = data)) 
    
    mean <- model$coefficients[names(model$coefficients) %>% str_detect(":")]
    ci_tbl <- model %>% confint() %>% as_tibble()
    row_names <- model %>% confint() %>% rownames()
    idx <- which(row_names %>% str_detect(":"))
    low <- ci_tbl$`2.5 %`[idx]
    high <- ci_tbl$`97.5 %`[idx]
    idx <- which(rownames(car::Anova(model, type = "III")) %>% str_detect(":"))
    p <- car::Anova(model, type = "III")[[4]][idx]
    sig = ""
    if (p < 0.05){sig = "*"}
    if (p < 0.01){sig = "**"}
    if (p < 0.001){sig = "***"}
    ci_summary <- paste0("mean = ", round(mean, 3), 
                         "; 95% CI = ", round(low, 3), ", ", round(high, 3), 
                         "; P = ", round(p, 4))
        
    return(list(
        "base_model" = base_model,        
        "model" = model,
        "lm_model" = lm_model,
        "ci_summary" = ci_summary))}

In [5]:
intention <- test_interaction_gov_trust(data = vcf_data %>% dplyr::mutate(post = intention_6_n, pre = base_intention_n))
encourage <- test_interaction_gov_trust(data = vcf_data %>% dplyr::mutate(post = encourage_6_n, pre = base_encourage_n))
sp_confidence <- test_interaction_gov_trust(data = vcf_data %>% dplyr::mutate(post = sp_confidence_6_n, pre = base_sp_confidence_n))
sp_complacency <- test_interaction_gov_trust(data = vcf_data %>% dplyr::mutate(post = sp_complacency_6_n, pre = base_sp_complacency_n))

In [6]:
paste0("Intentions: ", intention$ci_summary)
paste0("Intentions to encourage: ", encourage$ci_summary)
paste0("Vaccine confidence beliefs: ", sp_confidence$ci_summary)
paste0("Vaccine complacency beliefs: ", sp_complacency$ci_summary)

[1] "Intentions: mean = -0.004; 95% CI = -0.104, 0.097; P = 0.9447"

[1] "Intentions to encourage: mean = -0.022; 95% CI = -0.16, 0.116; P = 0.7528"

[1] "Vaccine confidence beliefs: mean = 0.064; 95% CI = -0.1, 0.228; P = 0.4412"

[1] "Vaccine complacency beliefs: mean = 0.024; 95% CI = -0.162, 0.209; P = 0.8028"

### Party Affiliation Interactions

In [7]:
test_interaction_republican <- function(data){
    
    data$group <- factor(data$group, levels = c("Control", "Treatment"))    
    
    # fit base model    
    base_model <- aov(post ~ group*republican + wave + 
                      phi_trust_cat + conservative +  
                      employment2 + education2 + marital_status + gender + age + income2 +                     
                      pre, data = data)   
     
    # calculate Box-Cox lambda
    bc <- trafo::boxcox(base_model, method = "ml", lambdarange = c(-10, 3), plotit = FALSE)
    lambda <- bc$lambdahat
    bctrans <- make.tran("boxcox", lambda)
  
    # fit model to transformed data    
    model <- with(bctrans,
              aov(linkfun(post) ~ group*republican + wave + 
                                  phi_trust_cat + conservative +
                                  employment2 + education2 + marital_status + gender + age + income2 +                     
                                  linkfun(pre), data = data))             
    
    lm_model <- with(bctrans,
              lm(linkfun(post) ~ group*republican + wave + 
                                  phi_trust_cat + conservative +
                                  employment2 + education2 + marital_status + gender + age + income2 +                     
                                  linkfun(pre), data = data))              
    
    mean <- model$coefficients[names(model$coefficients) %>% str_detect(":")]
    ci_tbl <- model %>% confint() %>% as_tibble()
    row_names <- model %>% confint() %>% rownames()
    idx <- which(row_names %>% str_detect(":"))
    low <- ci_tbl$`2.5 %`[idx]
    high <- ci_tbl$`97.5 %`[idx]
    idx <- which(rownames(car::Anova(model, type = "III")) %>% str_detect(":"))
    p <- car::Anova(model, type = "III")[[4]][idx]
    sig = ""
    if (p < 0.05){sig = "*"}
    if (p < 0.01){sig = "**"}
    if (p < 0.001){sig = "***"}
    ci_summary <- paste0("mean = ", round(mean, 3), 
                         "; 95% CI = ", round(low, 3), ", ", round(high, 3), 
                         "; P = ", round(p, 4))
        
    return(list(
        "base_model" = base_model,        
        "model" = model,
        "lm_model" = lm_model,
        "ci_summary" = ci_summary))}

In [8]:
intention <- test_interaction_republican(data = vcf_data %>% dplyr::mutate(post = intention_6_n, pre = base_intention_n))
encourage <- test_interaction_republican(data = vcf_data %>% dplyr::mutate(post = encourage_6_n, pre = base_encourage_n))
sp_confidence <- test_interaction_republican(data = vcf_data %>% dplyr::mutate(post = sp_confidence_6_n, pre = base_sp_confidence_n))
sp_complacency <- test_interaction_republican(data = vcf_data %>% dplyr::mutate(post = sp_complacency_6_n, pre = base_sp_complacency_n))

In [9]:
paste0("Intentions: ", intention$ci_summary)
paste0("Intentions to encourage: ", encourage$ci_summary)
paste0("Vaccine confidence beliefs: ", sp_confidence$ci_summary)
paste0("Vaccine complacency beliefs: ", sp_complacency$ci_summary)

[1] "Intentions: mean = -0.026; 95% CI = -0.129, 0.078; P = 0.6266"

[1] "Intentions to encourage: mean = -0.094; 95% CI = -0.235, 0.046; P = 0.1861"

[1] "Vaccine confidence beliefs: mean = -0.055; 95% CI = -0.22, 0.109; P = 0.507"

[1] "Vaccine complacency beliefs: mean = -0.071; 95% CI = -0.255, 0.113; P = 0.4491"

### Political Views Interactions

In [10]:
test_interaction_conservative <- function(data){
    
    data$group <- factor(data$group, levels = c("Control", "Treatment"))    
    
    # fit base model    
    base_model <- aov(post ~ group*conservative + wave + 
                      phi_trust_cat + republican +  
                      employment2 + education2 + marital_status + gender + age + income2 +                     
                      pre, data = data)
     
    # calculate Box-Cox lambda
    bc <- trafo::boxcox(base_model, method = "ml", lambdarange = c(-10, 3), plotit = FALSE)
    lambda <- bc$lambdahat
    bctrans <- make.tran("boxcox", lambda)
  
    # fit model to transformed data    
    model <- with(bctrans,
              aov(linkfun(post) ~ group*conservative + wave + 
                                  phi_trust_cat + republican +
                                  employment2 + education2 + marital_status + gender + age + income2 +                     
                                  linkfun(pre), data = data))     
    
    lm_model <- with(bctrans,
              lm(linkfun(post) ~ group*conservative + wave + 
                                  phi_trust_cat + republican +
                                  employment2 + education2 + marital_status + gender + age + income2 +                     
                                  linkfun(pre), data = data))     
    
    mean <- model$coefficients[names(model$coefficients) %>% str_detect(":")]
    ci_tbl <- model %>% confint() %>% as_tibble()
    row_names <- model %>% confint() %>% rownames()
    idx <- which(row_names %>% str_detect(":"))
    low <- ci_tbl$`2.5 %`[idx]
    high <- ci_tbl$`97.5 %`[idx]
    idx <- which(rownames(car::Anova(model, type = "III")) %>% str_detect(":"))
    p <- car::Anova(model, type = "III")[[4]][idx]
    sig = ""
    if (p < 0.05){sig = "*"}
    if (p < 0.01){sig = "**"}
    if (p < 0.001){sig = "***"}
    ci_summary <- paste0("mean = ", round(mean, 3), 
                         "; 95% CI = ", round(low, 3), ", ", round(high, 3), 
                         "; P = ", round(p, 4))
        
    return(list(
        "base_model" = base_model,        
        "model" = model,
        "lm_model" = lm_model, 
        "ci_summary" = ci_summary))}

In [11]:
intention <- test_interaction_conservative(data = vcf_data %>% dplyr::mutate(post = intention_6_n, pre = base_intention_n))
encourage <- test_interaction_conservative(data = vcf_data %>% dplyr::mutate(post = encourage_6_n, pre = base_encourage_n))
sp_confidence <- test_interaction_conservative(data = vcf_data %>% dplyr::mutate(post = sp_confidence_6_n, pre = base_sp_confidence_n))
sp_complacency <- test_interaction_conservative(data = vcf_data %>% dplyr::mutate(post = sp_complacency_6_n, pre = base_sp_complacency_n))

In [12]:
paste0("Intentions: ", intention$ci_summary)
paste0("Intentions to encourage: ", encourage$ci_summary)
paste0("Vaccine confidence beliefs: ", sp_confidence$ci_summary)
paste0("Vaccine complacency beliefs: ", sp_complacency$ci_summary)

[1] "Intentions: mean = -0.024; 95% CI = -0.124, 0.077; P = 0.6426"

[1] "Intentions to encourage: mean = 0.1; 95% CI = -0.037, 0.237; P = 0.1528"

[1] "Vaccine confidence beliefs: mean = 0.057; 95% CI = -0.103, 0.218; P = 0.4843"

[1] "Vaccine complacency beliefs: mean = 0.078; 95% CI = -0.102, 0.258; P = 0.3964"